## Subgraph
Give a graph $G=(V,E)$, either:
1. $G'$ is a subgraph of $G$ ***induced*** by $V'$, where $G'=(V',E')$ such that $V'\subseteq V$ and $E'=\{(u,v)\in E | u,v\in V'\}$
2. $G'$ is a subgraph of $G$ ***induced*** by $E'$, where $G'=(V',E')$ such that $E'\subseteq E$ and $V'=\{v\in V | (v,u)\in E'\}$